In [21]:
# Data available: https://www.ncdc.noaa.gov/cdo-web/search
# 1.csv Jan 1, 2013- April 5, 2022
# 2.csv Jan 1, 2003 – December 31, 2012 
# 3.csv Jan 1, 1993 - December 31, 2002 
# 4.csv Jan 1, 1983 - December 31, 1992 
# 5.csv Jan 1, 1973 - December 31, 1982 

In [22]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [23]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()

In [24]:
ncd_cols = ['DATE', 
            'HourlyAltimeterSetting', 'HourlyDryBulbTemperature', 
            'HourlyDewPointTemperature', 'HourlyRelativeHumidity', 
            'HourlyWindSpeed', 'HourlyWindDirection']

In [25]:
df1 = pd.read_csv('datasets/nenana_climate/1.csv', usecols = ncd_cols) 
df2 = pd.read_csv('datasets/nenana_climate/2.csv', usecols = ncd_cols) 
df3 = pd.read_csv('datasets/nenana_climate/3.csv', usecols = ncd_cols) 
df4 = pd.read_csv('datasets/nenana_climate/4.csv', usecols = ncd_cols) 
df5 = pd.read_csv('datasets/nenana_climate/5.csv', usecols = ncd_cols)  

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42,43,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42,43,44,49,57) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42,43,44,55) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_as

In [26]:
date_object = lambda d : datetime.strptime(d, '%Y-%m-%dT%H:%M:%S')
shortened_date = lambda d : datetime.strptime(d[:10], '%Y-%m-%d')
# to fix some numerical values saved as strings; some end with 's'
remove_s = lambda n : (float(n[:-1]) if n.endswith('s') else float(n)) if type(n)== str else n

In [27]:
df1 = df1.dropna()
df1['date_obj']= df1['DATE'].apply(date_object)
df1['short_day']= df1['DATE'].apply(shortened_date)
df1['HourlyDewPointTemperature'] = df1['HourlyDewPointTemperature'].apply(remove_s)
df1['HourlyAltimeterSetting']   = df1['HourlyAltimeterSetting'].apply(remove_s)
df1['HourlyDryBulbTemperature'] = df1['HourlyDryBulbTemperature'].apply(remove_s)

In [28]:
df2 = df2.dropna()
df2['date_obj']= df2['DATE'].apply(date_object)
df2['short_day']= df2['DATE'].apply(shortened_date)
# this dataset has some random astericks for values 
df2 = df2.replace('*', pd.NA) 
df2 = df2.dropna()
df2['HourlyDewPointTemperature'] = df2['HourlyDewPointTemperature'].apply(remove_s)
df2['HourlyAltimeterSetting']   = df2['HourlyAltimeterSetting'].apply(remove_s)
df2['HourlyDryBulbTemperature'] = df2['HourlyDryBulbTemperature'].apply(remove_s)

In [29]:
df3 = df3.dropna()
df3['date_obj']= df3['DATE'].apply(date_object)
df3['short_day']= df3['DATE'].apply(shortened_date)
df3['HourlyDewPointTemperature'] = df3['HourlyDewPointTemperature'].apply(remove_s)
df3['HourlyAltimeterSetting']   = df3['HourlyAltimeterSetting'].apply(remove_s)
df3['HourlyDryBulbTemperature'] = df3['HourlyDryBulbTemperature'].apply(remove_s)

In [30]:
df4 = df4.dropna()
df4['date_obj']= df4['DATE'].apply(date_object)
df4['short_day']= df4['DATE'].apply(shortened_date)
df4['HourlyDewPointTemperature'] = df4['HourlyDewPointTemperature'].apply(remove_s)
df4['HourlyAltimeterSetting']   = df4['HourlyAltimeterSetting'].apply(remove_s)
df4['HourlyDryBulbTemperature'] = df4['HourlyDryBulbTemperature'].apply(remove_s)

In [31]:
df5 = df5.dropna()
df5['date_obj']= df5['DATE'].apply(date_object)
df5['short_day']= df5['DATE'].apply(shortened_date)
df5['HourlyDewPointTemperature'] = df5['HourlyDewPointTemperature'].apply(remove_s)
df5['HourlyAltimeterSetting']   = df5['HourlyAltimeterSetting'].apply(remove_s)
df5['HourlyDryBulbTemperature'] = df5['HourlyDryBulbTemperature'].apply(remove_s)

In [32]:
dailies = pd.concat([df1.groupby('short_day').mean(), 
        df2.groupby('short_day').mean(), df3.groupby('short_day').mean(), 
        df4.groupby('short_day').mean(), df5.groupby('short_day').mean()])

In [33]:
dailies.index.min(), dailies.index.max()

(Timestamp('1973-01-05 00:00:00'), Timestamp('2022-01-01 00:00:00'))

In [34]:
dailies['Year'] = dailies.index.year

In [35]:
dailies = dailies.reset_index()

In [36]:
dailies.to_sql('daily_climate', conn, if_exists='replace', index = False)
dailies.to_csv('daily_climate.csv', index=False)

In [37]:
dailies

,short_day,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyWindSpeed,HourlyWindDirection,Year
0,2013-01-01,29.340833,16.333333,30.625000,55.625000,NaN,NaN,2013
1,2013-01-02,29.359167,2.958333,8.416667,78.625000,NaN,NaN,2013
2,2013-01-03,29.560417,13.375000,27.916667,54.541667,NaN,NaN,2013
3,2013-01-04,29.714167,2.375000,10.958333,69.375000,NaN,NaN,2013
4,2013-01-05,29.543333,4.875000,16.416667,60.958333,NaN,NaN,2013
...,...,...,...,...,...,...,...,...
15032,1982-12-27,30.000000,5.333333,14.000000,69.000000,11.333333,166.666667,1982
15033,1982-12-28,30.000000,13.000000,18.000000,81.000000,89.000000,330.000000,1982
15034,1982-12-29,30.000000,24.400000,39.000000,57.000000,3.600000,125.000000,1982
15035,1982-12-30,30.174583,27.250000,30.166667,89.333333,3.166667,152.500000,1982


In [38]:
events = pd.read_sql("""
    SELECT * 
    from nib;
    """, con=conn)
events

,Year,Decimal Day of Year,Date Time,month,TOD,TOY
0,1917,120.4792,04/30/1917 11:30:00 AM,04,0.4792,120.0
1,1918,131.3979,05/11/1918 09:33:00 AM,05,0.3979,131.0
2,1919,123.6063,05/03/1919 02:33:00 PM,05,0.6063,123.0
3,1920,132.4486,05/11/1920 10:46:00 AM,05,0.4486,132.0
4,1921,131.2792,05/11/1921 06:42:00 AM,05,0.2792,131.0
...,...,...,...,...,...,...
99,2016,114.6521,04/23/2016 03:39:00 PM,04,0.6521,114.0
100,2017,121.5000,05/01/2017 12:00:00 PM,05,0.5000,121.0
101,2018,121.5542,05/01/2018 01:18:00 PM,05,0.5542,121.0
102,2019,104.0146,04/14/2019 12:21:00 AM,04,0.0146,104.0


In [39]:
features = pd.read_sql("""
    SELECT * 
    from daily_climate
    inner join nib using(Year);
    """, con=conn)
features

,short_day,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,HourlyWindSpeed,HourlyWindDirection,Year,Decimal Day of Year,Date Time,month,TOD,TOY
0,2013-01-01 00:00:00,29.340833,16.333333,30.625000,55.625000,NaN,NaN,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
1,2013-01-02 00:00:00,29.359167,2.958333,8.416667,78.625000,NaN,NaN,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
2,2013-01-03 00:00:00,29.560417,13.375000,27.916667,54.541667,NaN,NaN,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
3,2013-01-04 00:00:00,29.714167,2.375000,10.958333,69.375000,NaN,NaN,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
4,2013-01-05 00:00:00,29.543333,4.875000,16.416667,60.958333,NaN,NaN,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14669,1982-12-27 00:00:00,30.000000,5.333333,14.000000,69.000000,11.333333,166.666667,1982,130.7333,05/10/1982 05:36:00 PM,05,0.7333,130.0
14670,1982-12-28 00:00:00,30.000000,13.000000,18.000000,81.000000,89.000000,330.000000,1982,130.7333,05/10/1982 05:36:00 PM,05,0.7333,130.0
14671,1982-12-29 00:00:00,30.000000,24.400000,39.000000,57.000000,3.600000,125.000000,1982,130.7333,05/10/1982 05:36:00 PM,05,0.7333,130.0
14672,1982-12-30 00:00:00,30.174583,27.250000,30.166667,89.333333,3.166667,152.500000,1982,130.7333,05/10/1982 05:36:00 PM,05,0.7333,130.0


In [40]:
[(f, len(features[features[f].isnull()])) for f in features.columns]

[('short_day', 0),
 ('HourlyAltimeterSetting', 0),
 ('HourlyDewPointTemperature', 0),
 ('HourlyDryBulbTemperature', 0),
 ('HourlyRelativeHumidity', 3247),
 ('HourlyWindSpeed', 6156),
 ('HourlyWindDirection', 8472),
 ('Year', 0),
 ('Decimal Day of Year', 0),
 ('Date Time', 0),
 ('month', 0),
 ('TOD', 0),
 ('TOY', 0)]

In [41]:
features['TOY']

0        140.0
1        140.0
2        140.0
3        140.0
4        140.0
         ...  
14669    130.0
14670    130.0
14671    130.0
14672    130.0
14673    130.0
Name: TOY, Length: 14674, dtype: float64

In [42]:
dataset1= features.drop(['HourlyWindSpeed','HourlyWindDirection','Year', 'Decimal Day of Year', 'Date Time',
       'TOD'],axis=1)
dataset1 = dataset1[(dataset1['HourlyRelativeHumidity'].notna())]
numerical_features=dataset1.drop(['TOY'], axis=1)
numerical_features = numerical_features.reset_index(drop=True)

In [43]:
numerical_features

,short_day,HourlyAltimeterSetting,HourlyDewPointTemperature,HourlyDryBulbTemperature,HourlyRelativeHumidity,month
0,2013-01-01 00:00:00,29.340833,16.333333,30.625000,55.625000,05
1,2013-01-02 00:00:00,29.359167,2.958333,8.416667,78.625000,05
2,2013-01-03 00:00:00,29.560417,13.375000,27.916667,54.541667,05
3,2013-01-04 00:00:00,29.714167,2.375000,10.958333,69.375000,05
4,2013-01-05 00:00:00,29.543333,4.875000,16.416667,60.958333,05
...,...,...,...,...,...,...
11422,1982-12-27 00:00:00,30.000000,5.333333,14.000000,69.000000,05
11423,1982-12-28 00:00:00,30.000000,13.000000,18.000000,81.000000,05
11424,1982-12-29 00:00:00,30.000000,24.400000,39.000000,57.000000,05
11425,1982-12-30 00:00:00,30.174583,27.250000,30.166667,89.333333,05


In [44]:
numerical_features.to_csv('numerical_features.csv', index=False)